## Лабораторная работа №2
по дисциплине *Data Mining и исследование данных*

Вариант 4

Работа выполнена **Воропаевым В.С.**, гр. **М1О-415Бки-19**

Цель лабораторной работы: разработать модели для определения качества вин, подобрать гиперпараметры и сохранить их вместе с обученными моделями.

Для начала сделаем pipeline для предобработки. Предобработка будет состоять из следующих этапов:
 - для всех столбцов кроме ```quality``` - привести значения к одному диапазону
 - для столбца ```quality``` - привести значения к двум классам - хорошему и плохому вину (уровень качества вина строго больше/равен или меньше 5 соответственно)

In [91]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

class Categorizer(BaseEstimator, TransformerMixin):
    def fit(self, x, y = None):
        return self

    def transform(self, X, y = None):   #Определяем вино как плохое или хорошее
        res = [1 if int(x) > 5 else 0 for x in X.values]
        return pd.DataFrame(res)

    def get_feature_names_out(self, input_features=None):   #Оставить имена колонок в исходном виде
        return input_features

StandardScaler().get_feature_names_out

red_dataset = pd.read_csv('datasets/winequality-red.csv', sep=';')

non_quality = [col for col in red_dataset.columns if col != 'quality']

pipeline = ColumnTransformer(
    [('category', Categorizer(), ['quality'])],
    remainder=(StandardScaler()),    #Остальные столбцы масштабируем
    verbose_feature_names_out=False)    #Не добавляем префикс к названиям столбцов 
pipeline.set_output(transform = 'pandas')


red_dataset = pipeline.fit_transform(red_dataset)

В результате получаем данные в гораздо более близких диапазонах друг к другу, чем раньше. Далее, перед созданием и обучением моделей необходимо разделить датасет на тестовую и обучающую выборку, а из каждой выборки в свою очередь убрать значения качества вина, чтобы они не участвовали в процессе обучения в качестве параметра. Разделение на выборки лучше проводить с помощью ```StratifiedShuffleSplit```, чтобы выборки содержали в себе различные образцы, а не схожие (иначе модели переобучатся, и не смогут распознавать другие образцы).

In [92]:
from sklearn.model_selection import StratifiedShuffleSplit

red_labels = red_dataset['quality']
red_data = red_dataset.copy().drop('quality', axis=1)

str_shuf_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=2022)
for train, test in str_shuf_split.split(red_data, red_labels):
    pass

red_X_test = red_data.iloc[test]
red_y_test = red_labels.iloc[test]

red_X_train = red_data.iloc[train]
red_y_train = red_labels.iloc[train]

Аналогично для белых вин:

In [93]:
white_dataset = pd.read_csv('datasets/winequality-white.csv', sep=';')

white_dataset = pipeline.fit_transform(white_dataset)

white_labels = white_dataset['quality']
white_data = white_dataset.drop('quality', axis=1)

str_shuf_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=2022)
for train, test in str_shuf_split.split(white_data, white_labels):
    pass

white_X_test = white_data.iloc[test]
white_y_test = white_labels.iloc[test]

white_X_train = white_data.iloc[train]
white_y_train = white_labels.iloc[train]

white_dataset

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,1.0,0.172097,-0.081770,0.213280,2.821349,-0.035355,0.569932,0.744565,2.331512,-1.246921,-0.349184,-1.393152
1,1.0,-0.657501,0.215896,0.048001,-0.944765,0.147747,-1.253019,-0.149685,-0.009154,0.740029,0.001342,-0.824276
2,1.0,1.475751,0.017452,0.543838,0.100282,0.193523,-0.312141,-0.973336,0.358665,0.475102,-0.436816,-0.336667
3,1.0,0.409125,-0.478657,-0.117278,0.415768,0.559727,0.687541,1.121091,0.525855,0.011480,-0.787342,-0.499203
4,1.0,0.409125,-0.478657,-0.117278,0.415768,0.559727,0.687541,1.121091,0.525855,0.011480,-0.787342,-0.499203
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,1.0,-0.776015,-0.677101,-0.365197,-0.944765,-0.310008,-0.664970,-1.091000,-0.965483,0.541334,0.088973,0.557282
4894,0.0,-0.301959,0.414339,0.213280,0.317179,0.056196,1.275590,0.697499,0.291789,-0.253446,-0.261553,-0.743008
4895,1.0,-0.420473,-0.379435,-1.191592,-1.023637,-0.218457,-0.312141,-0.643875,-0.497350,-1.313153,-0.261553,-0.905544
4896,1.0,-1.605613,0.116674,-0.282557,-1.043355,-1.088192,-0.900190,-0.667408,-1.784717,1.004955,-0.962605,1.857572


Далее приступим к обучению, подбору гиперпараметров, тестированию и оценке моделей. Так как шаги для каждой модели одинаковы, то будет удобнее выделмть все эти действия в одну функцию:

In [94]:
import pickle
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings(action='ignore', category=ConvergenceWarning)


def learn_and_score(colorprefix, x_train: pd.core.frame.DataFrame, y_train: pd.core.series.Series, x_test, y_test, pipeline):
    score = pd.DataFrame({'scores' : ['Accuracy', 'Recall', 'Precision', 'ROC AUC curve']})
    classifiers = {         #Словарь классификаторов и их параметров
        'logistic-regression': (LogisticRegression(solver='liblinear', max_iter=250, penalty='l2'),
                                    {'clf__C': uniform(loc=0.01, scale=2), 
                                    'clf__tol': uniform(loc=1e-8, scale=1e-2)}),
        'SVM':                  (SVC(kernel='linear', max_iter=250),
                                    {'clf__C': uniform(loc=0.01, scale=2),
                                    # 'clf__gamma': uniform(loc=0, scale=10), 
                                    # 'clf__degree': randint(2,5), 
                                    'clf__tol': uniform(loc=1e-8, scale=1e-2)}),
        'KNN':                  (KNeighborsClassifier(algorithm='kd_tree'),
                                    {'clf__n_neighbors': randint(3, 8), 
                                    'clf__leaf_size': randint(10, 60)}),
        'naive-bayes':          (GaussianNB(),
                                    {'clf__var_smoothing': uniform(loc=1e-10, scale=1)}),
        'random-forest':        (RandomForestClassifier(random_state=2022),
                                    {'clf__n_estimators': randint(100, 300), 
                                    'clf__max_features': ['sqrt', 'log2'],
                                    'clf__min_samples_split': randint(2, 5)})
    }


    for name, values in classifiers.items():
        clf, params = values
        pipeline = Pipeline([
            ('clf', clf)]
        )
        model = RandomizedSearchCV(pipeline, params)
        model.fit(x_train, y_train)

        with open(f"models/{colorprefix}wine/{name}.pkl", "wb") as file:
            pickle.dump(model, file, protocol=3)
        with open(f"models/{colorprefix}wine/{name}.txt", "w") as file:
            file.write(str(model.best_params_))
        print(colorprefix + '|' + name)
        print(model.best_params_, '\n' + '=' * 30)

        y_pred = model.predict(x_test)
        y_true = np.array(y_test)

        score[name] = [accuracy_score(y_true, y_pred), recall_score(y_true, y_pred),
                        precision_score(y_true, y_pred), roc_auc_score(y_true, y_pred)]

        print(f'Confusion matrix:\n {confusion_matrix(y_true, y_pred)}\n')
    return score

learn_and_score('red', red_X_train, red_y_train, red_X_test, red_y_test, pipeline)


red|logistic-regression
{'clf__C': 1.0188721901979945, 'clf__tol': 0.009227130288899755} 
Confusion matrix:
 [[117  32]
 [ 44 127]]

red|SVM
{'clf__C': 1.4543183607059906, 'clf__tol': 0.0038546722586255684} 
Confusion matrix:
 [[ 54  95]
 [ 22 149]]

red|KNN
{'clf__leaf_size': 32, 'clf__n_neighbors': 6} 
Confusion matrix:
 [[106  43]
 [ 56 115]]

red|naive-bayes
{'clf__var_smoothing': 0.45070526279411205} 
Confusion matrix:
 [[111  38]
 [ 45 126]]

red|random-forest
{'clf__max_features': 'log2', 'clf__min_samples_split': 3, 'clf__n_estimators': 267} 
Confusion matrix:
 [[119  30]
 [ 30 141]]



,scores,logistic-regression,SVM,KNN,naive-bayes,random-forest
0,Accuracy,0.762500,0.634375,0.690625,0.740625,0.812500
1,Recall,0.742690,0.871345,0.672515,0.736842,0.824561
2,Precision,0.798742,0.610656,0.727848,0.768293,0.824561
3,ROC AUC curve,0.763962,0.616881,0.691962,0.740904,0.811610


In [95]:
learn_and_score('white', white_X_train, white_y_train, white_X_test, white_y_test, pipeline)

white|logistic-regression
{'clf__C': 0.40971197071721055, 'clf__tol': 0.003232279771915137} 
Confusion matrix:
 [[155 173]
 [ 87 565]]

white|SVM
{'clf__C': 0.8351398010402713, 'clf__tol': 0.00459449461206454} 
Confusion matrix:
 [[104 224]
 [147 505]]

white|KNN
{'clf__leaf_size': 52, 'clf__n_neighbors': 7} 
Confusion matrix:
 [[165 163]
 [ 93 559]]

white|naive-bayes
{'clf__var_smoothing': 0.09170029411973789} 
Confusion matrix:
 [[185 143]
 [144 508]]

white|random-forest
{'clf__max_features': 'sqrt', 'clf__min_samples_split': 3, 'clf__n_estimators': 274} 
Confusion matrix:
 [[229  99]
 [ 64 588]]



,scores,logistic-regression,SVM,KNN,naive-bayes,random-forest
0,Accuracy,0.734694,0.621429,0.738776,0.707143,0.833673
1,Recall,0.866564,0.774540,0.857362,0.779141,0.901840
2,Precision,0.765583,0.692730,0.774238,0.780338,0.855895
3,ROC AUC curve,0.669563,0.545807,0.680205,0.671583,0.800006


В итоге для обоих датасетов лучший результат показало применение случайного леса. С другой стороны, именно его обучение занимает больше всего времени. Для белого вина случайный лес показал лучшие результаты по всем параметрам, однако для красного метод SVM показал лучший Recall - что показывает, что модель в данной ситуации с большей вероятностью выберет правильно хорошие вина, но при этом в эту выборку могут попасть и плохие.